In [33]:
import pandas as pd

In [34]:
df=pd.read_csv("C:/Users/olw09/karthikcase/descrete-optimization/food/outputs/quantity.csv")
df

,Dish,Hour,Requirement,Prepare,Inventory,Waste,Unfilled,Sold
0,Samosa,0,0,0.0,0.0,0.0,0.0,0.0
1,Samosa,1,0,0.0,0.0,0.0,0.0,0.0
2,Samosa,2,0,0.0,0.0,0.0,0.0,0.0
3,Samosa,3,0,0.0,0.0,0.0,0.0,0.0
4,Samosa,4,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
275,ice cream,9,198,0.0,634.0,0.0,0.0,198.0
276,ice cream,10,218,0.0,436.0,0.0,0.0,218.0
277,ice cream,11,210,0.0,218.0,0.0,0.0,210.0
278,ice cream,12,190,190.0,8.0,0.0,0.0,190.0


In [35]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("C:/Users/olw09/karthikcase/descrete-optimization/food/outputs/quantity.csv")

# Save the DataFrame to an Excel file
df.to_excel("C:/Users/olw09/karthikcase/descrete-optimization/food/outputs/quantity.xlsx", index=False)
